### Updating The FX_derivatives_all database 
Update with the TJ06 and TX06sap reports.

In [ ]:
#Imports the daily TJ06 (options) & TX06 (forwards) and updates FX_Derivatives_All Database 
#which is located in the Financial_Database directory located within the Treasury drive

import pandas as pd
import csv
import datetime

#this function module returns the delta days for file date (ie. If today is Monday, we need Friday's date
#for the SAP file)
def whatday():
    wkday = datetime.datetime.today().weekday()
    if wkday == 0:    
        return 3
    else:
        return 1

#this function module converts floats to strings.
def change2str(df, column):
    df[column] = df[column].astype(int)
    df[column] = df[column].astype(str)

#this function inverts JPY rate    
def buy_calc(rate,currency):
    if currency == 'JPY':
        buy_rate = 1/rate
    else:
        buy_rate = rate
    return buy_rate
    
    
#Putting into a variable date to be used with the file names
today = datetime.date.today()
prevday = today - datetime.timedelta(days=whatday())
prevday = prevday.strftime('%m%d%Y')

#Build file paths to get the daily TJ06 (options) and TX06 (forwards) files. 
dir_path = r'\\or1isi1vl3921nfs\sapnfsc\PRD003\0256'
file1_name = '\TX06_Extract' + prevday + '.txt'
file2_name = '\TJ06_Extract' + prevday + '.txt'
file_path1 = dir_path + file1_name
file_path2 = dir_path + file2_name

#Formating the header for daily TJ06 (options) and TX06 (forwards) files.
df1 = pd.read_csv(file_path1, delimiter = '|', skiprows=[0,2], header =[0])
df2 = pd.read_csv(file_path2, delimiter = '|', header =[0])

#Aligned header to match data columns for TJ06 (Option) report and removed blank spaces on header fields.
names = list(df2.columns.values)
newname = []
for name in names:
    newname.append(name.strip())
newname.append(newname.pop(0))
df2.columns = newname
df2.reset_index()


df2.insert(0,"Entity", 'ADIR')
df2.dropna(subset = ["Transactn"], inplace=True)

df2.reset_index(drop=True, inplace=True)

df2.dropna(axis=1, how='all', inplace=True)

#create dictionary of counter party names and their associated numbers
counter_party_dicts = {'61':'JPM',
                       '2':'BOA',
                       '2081':'USB',
                       '2121':'WFB',
                       '2101':'HSBC',
                       '12600': 'SG',
                       '1200000141': 'BOA',
                       '2131':'MSC',
                       '3':'UBS',
                       '5':'GS',
                       '72':'CITI',
                       '10020':'RBS'}

#This changes format by using the change2str function.
change2str(df2, 'Counterparty')
change2str(df2, 'Transactn')   

#This converts the SAP BP code to a counterparty name.
i=0
for counterparties in df2.loc[:,'Counterparty']:
    
    df2.loc[i,'Counterparty'] = counter_party_dicts[counterparties]
    i+=1

#Creating dictionary to add currency pair
currencypair_dict = {'EUR':'EURUSD',
                    'GBP':'GBPUSD',
                    'AUD':'AUDUSD',
                    'JPY':'USDJPY',
                    'INR':'USDINR'}    

#Add additional columns
df2.insert(0, 'ID',"ADIR" + df2['Transactn'])

#Removing unwanted columns
df2.drop(columns=["Type", "Exercise", "Settlement", "Premium Curr", "Transactn"], inplace= True)



#Add additional columns
df2['Currency_Pair'], df2['Buy_Currency'], df2['Sell_Currency'], df2['Buy_Amount'] = ["", "USD", "", 0]

#Split underlying column into sell amount and sell currency amount.
for index, amount in enumerate(df2['Underlying']):
    Curr, Amt = amount.strip().split(' ')
    
    df2.loc[index,'Sell_Currency'] = Curr
    df2.loc[index,'Currency_Pair'] = currencypair_dict[Curr]
    
#Adding Buy Amount buy multiplying the strike and underlying. Also trancating the Buy_Amount to two decimals
for index, amount in enumerate(df2['Underlying']):
    Curr, Amt = amount.strip().split(' ')
    Amt = float(Amt.replace(',','').strip('-'))
    df2.loc[index,'Buy_Amount'] = Amt*buy_calc(df2.loc[index,'Strike'], Curr)
    df2.loc[index,'Underlying'] = Amt*-1
    
#Rename columns to match FX Derivatives all
df2.rename(columns = {"Premium Amt":"Premium", 
                      "Term Start":"Trade_Date", 
                      "Due Date":"Expiry_Date", 
                      "Strike":"Contract_Rate",
                      "Underlying":"Sell_Amount"},inplace=True)


#reordering column names
df2 = df2[['ID',
           'Entity',
           'Counterparty',
           'Currency_Pair',
           'Buy_Currency',
           'Sell_Currency',
           'Trade_Date',
           'Expiry_Date',
           'Contract_Rate',
           'Buy_Amount',
           'Sell_Amount',
           'Premium']]



In [ ]:
#Build file paths to get FX database

from openpyxl import Workbook
book = Workbook('r\\sjshare\finance\Treasury\Financial_Database\test\FX_Derivatives_Alltest.xlsx',
               sheet_name ='FX_options_static')
                
for key  in key:
    keys = df2('ID')
    sheet.append(row)
                
                
workbook.save
#workbook.save(filename="FX_Derivatives_Alltest.xlsx")          



In [ ]:
import openpyxl
output_sheet = r'FX_options_static'
output_file = r'\\sjshare\finance\Treasury\Financial_Database\test\FX_Derivatives_Alltest.xlsx'
book = openpyxl.load_workbook(output_file)
writer = pd.ExcelWriter(output_file, engine='openpyxl')
writer.book = book
writer.sheets = output_sheet
   
# iterate through dictionary and output dataframes to worksheets
df2.to_excel(writer,
    sheet_name=output_sheet,
    startrow=800,
    startcol=0,
    header=False,
    index=False)


writer.save()
writer.close()

In [ ]:
import openpyxl
output_sheet = r'FX_options_panda'
output_file = r'C:\Users\chtyler\Desktop\FX_Derivatives_Alltest.xlsx'
#book = openpyxl.load_workbook(output_file)
#writer = pd.ExcelWriter(output_file, engine='openpyxl')
#writer.book = book
#writer.sheets = output_sheet

with pd.ExcelWriter(output_file, engine='openpyxl', index=False, header=False, mode='a') as writer:
    df2.to_excel(writer,sheet_name=output_sheet)
    writer.save()
writer.close()

In [ ]:
#trying to get rid of scientific notation

#df2.apply(lambda x: '%.5f' % x, axis=1)

#df2["Buy_Amount"].describe().apply(lambda x: format(x, 'f'))

df2.to_excel(r'SAP_Options.xlsx', index = False)

#df2['Buy_Amount'].(lambda s: s.apply(lambda x: format(x, 'f')))

In [ ]:
df2